In [7]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install transformers torch accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

class Agent:
    def __init__(
        self,
        role,
        goal,
        backstory,
        allow_delegation=False,
        verbose=True,
        model_name="EleutherAI/gpt-neo-1.3B"
    ):
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.allow_delegation = allow_delegation
        self.verbose = verbose

        # Load the GPT-Neo model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

    def generate_response(self, prompt):
        # Tokenize the input prompt
        inputs = self.tokenizer(prompt, return_tensors="pt")

        # Generate a response
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=100,  # Adjust max_length as needed
                num_return_sequences=1,
                pad_token_id=self.tokenizer.eos_token_id
            )

        # Decode the generated text
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    def act(self, customer):
        # Format the backstory with the customer name
        formatted_backstory = self.backstory.format(customer=customer)

        # Creating the prompt for the model
        prompt = (
            f"Role: {self.role}\n"
            f"Goal: {self.goal}\n"
            f"Backstory: {formatted_backstory}\n"
            f"Customer: {customer}\n"
            "How can I assist you today?"
        )

        # Generate a response
        response = self.generate_response(prompt)

        if self.verbose:
            print(f"Agent ({self.role}) Response:\n{response}")

        return response


In [14]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        "are now working on providing support to {customer}, "
        "a super important customer for your company. "
        "You need to make sure that you provide the best support! "
        "Make sure to provide full complete answers, and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True
)

In [15]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [16]:
# Simulate a customer interaction
customer = "Acme Corp"
response = support_agent.act(customer)
print(response)

Agent (Senior Support Representative) Response:
Role: Senior Support Representative
Goal: Be the most friendly and helpful support representative in your team
Backstory: You work at crewAI (https://crewai.com) and are now working on providing support to Acme Corp, a super important customer for your company. You need to make sure that you provide the best support! Make sure to provide full complete answers, and make no assumptions.
Customer: Acme Corp
How can I assist you today?

I need a
Role: Senior Support Representative
Goal: Be the most friendly and helpful support representative in your team
Backstory: You work at crewAI (https://crewai.com) and are now working on providing support to Acme Corp, a super important customer for your company. You need to make sure that you provide the best support! Make sure to provide full complete answers, and make no assumptions.
Customer: Acme Corp
How can I assist you today?

I need a


In [17]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [18]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)


Creating Task

In [21]:
def inquiry_resolution(customer, inquiry, person):
    """
    Handles customer inquiries using the GPT-Neo-1.3B model.

    Args:
        customer (str): The customer's name or company.
        inquiry (str): The customer's inquiry.
        person (str): The person who made the inquiry.

    Returns:
        str: The AI-generated response.
    """

    # Format the inquiry for the model
    prompt = (
        f"{customer} just reached out with a super important ask:\n"
        f"{inquiry}\n\n"
        f"{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know "
        "to provide the best support possible. "
        "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    )

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate AI response
    output = model.generate(**inputs, max_length=300)

    # Decode and return the generated text
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage
customer = "TechCorp"
inquiry = "We need help integrating your API with our system. Can you provide documentation?"
person = "Alice Johnson"

response = inquiry_resolution(customer, inquiry, person)
print("\n💡 GPT-Neo Response:\n", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



💡 GPT-Neo Response:
 TechCorp just reached out with a super important ask:
We need help integrating your API with our system. Can you provide documentation?

Alice Johnson from TechCorp is the one that reached out. Make sure to use everything you know to provide the best support possible. You must strive to provide a complete and accurate response to the customer's inquiry.

We are looking for a full-time developer to help us build a new API for our customers. The API will be used to provide a variety of services to our customers.

The API will be used to provide a variety of services to our customers.

The API will be used to provide a variety of services to our customers.

The API will be used to provide a variety of services to our customers.

The API will be used to provide a variety of services to our customers.

The API will be used to provide a variety of services to our customers.

The API will be used to provide a variety of services to our customers.

The API will be used to